In [17]:
import torch
from torch.utils.data import DataLoader
from cellshape_cloud import CloudAutoEncoder
import cellshape_cloud as cloud
import cellshape_cluster as cluster
from cellshape_cloud.vendor.chamfer_distance import ChamferLoss
import os
#try chamfer loss


In [ ]:

from torch.utils.data import DataLoader
import cellshape_cloud as cloud


model = cloud.CloudAutoEncoder(num_features=128, 
                         k=40,
                         encoder_type="dgcnn",
                         decoder_type="foldingnet")
checkpoint = torch.load('testmodel_100epoch\model_output\save\model.pth')
model.load_state_dict(checkpoint['model_state_dict'])

In [6]:
import torch

# Check if CUDA is available
cuda_available = torch.cuda.is_available()

# Get the name of the GPU
gpu_name = torch.cuda.get_device_name(0) if cuda_available else "No GPU available"

print("CUDA available:", cuda_available)
print("GPU Name:", gpu_name)

CUDA available: True
GPU Name: NVIDIA GeForce RTX 3090


In [18]:
from cellshape_cluster import DeepEmbeddedClustering

dataset_dir = "data/pointcloud/"
autoencoder_model = "model_100epoch/model_output/save/model.pth"

num_features = 128
k = 40
encoder_type = "dgcnn"
num_clusters = 10
num_epochs = 50
learning_rate = 0.00001
gamma = 1
divergence_tolerance = 0.01
batch_size = 16
output_dir = "cluster/output/"
update_interval = 1

autoencoder = CloudAutoEncoder(
    num_features=128, 
    k=k, 
    encoder_type="dgcnn"
)

checkpoint = torch.load(autoencoder_model)

autoencoder.load_state_dict(checkpoint['model_state_dict'])

model = DeepEmbeddedClustering(autoencoder=autoencoder, 
                               num_clusters=10,
                               )


dataset = cloud.PointCloudDataset(dataset_dir)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False) # it is very important that shuffle=False here!
dataloader_inf = DataLoader(dataset, batch_size=1, shuffle=False) 


optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate * 16 / batch_size,
    betas=(0.9, 0.999),
    weight_decay=1e-6,
)

reconstruction_criterion = ChamferLoss()
cluster_criterion = torch.nn.KLDivLoss(reduction="sum")


name_logging = output_dir + "logging/directory"
name_model = output_dir + "save/model.pth"
name_writer = output_dir + "tensorboard/logs"
name = "my_experiment"
# Make sure directories exist
os.makedirs(name_logging, exist_ok=True)
os.makedirs(os.path.dirname(name_model), exist_ok=True)  # Since name_model includes the filename
os.makedirs(name_writer, exist_ok=True)

logging_info = (name_logging, name_model, name_writer, name)

cluster.train(
    model,
    dataloader,
    dataloader_inf,
    num_epochs,
    optimizer,
    reconstruction_criterion,
    cluster_criterion,
    update_interval,
    gamma,
    divergence_tolerance,
    logging_info
)

Performing k-means to get initial cluster centres
Initialising target distribution
Training epoch 0


Epoch 0: 100%|██████████| 58/58 [01:19<00:00,  1.37s/batch, clu_loss=0.847, rec_loss=6.62, tot_loss=7.46]
